# Exam 1

## Jiaqi Ma

## USC ID: 3028009001

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def mean_absolute_percentage_error(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    return 100 * np.mean(np.abs((y - y_pred) / y))

In [ ]:
df = pd.read_excel('sce.xlsx')

In [ ]:
df[:5]

1. (10 pts.) Use df['year'] = df['Date'].dt.year to create a column for the year. Repeat this step to create columns for the month, day, and hour. Also, use df['day of week'] = df['Date'].dt.dayofweek to create a column for the day of the week. Fit a multiple regression model with all predictors (new created columns) and with month, day, hour, day of week as categorical variables. Report the MAPE and r-square.

In [ ]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['hour'] = df['Date'].dt.hour
df['day_of_week'] = df['Date'].dt.dayofweek

In [ ]:
df[:5]

In [ ]:
df.dtypes

In [ ]:
df[['month','day','hour','day_of_week']] = df[['month','day','hour','day_of_week']].astype(object)

In [ ]:
df.dtypes

In [ ]:
model1 = smf.ols(formula = 'Load ~ year + C(month) + C(day) + C(hour) + C(day_of_week)',
                 data = df).fit()

In [ ]:
y1 = df['Load']
X1 = df[['year','month','day','hour','day_of_week']]

In [ ]:
y_pred1 = model1.predict(X1)

In [ ]:
# MAPE
mean_absolute_percentage_error(y1, y_pred1)

In [ ]:
# R-squared
model1.rsquared

2. Plot of Load as a function of time. This is a line plot (with the x-axis displaying the years). Column Date should be the dataframe index. Use df.Load.plot().

In [ ]:
df = df.set_index(df['Date'])
df = df.drop(columns = 'Date')
df[:5]

In [ ]:
df.Load.plot(figsize = (18,6))
plt.xlabel('Time')
plt.ylabel('Load')
plt.grid()

3. (20 pts.) Construct a scatterplot of Load vs temp with the x-axis displaying the temp. Add a quadratic least squares (red) line on the plot. This plot should suggest that the square of temp is a good predictor of the Load and that it should be included in the following models.

In [ ]:
df3 = df.sort_values('temp')
df3[:5]

In [ ]:
Load = df['Load']
temp = df['temp']
temp1 = temp.values.reshape(-1, 1)

In [ ]:
poly2 = PolynomialFeatures(degree = 2)
temp2 = poly2.fit_transform(temp1)

In [ ]:
model2 = LinearRegression().fit(temp2, Load)
yhat2 = model2.predict(temp2)

In [ ]:
plt.scatter(temp, Load, c = 'b', s =1)
plt.plot(temp, yhat2, c = 'r', lw = 1)
plt.xlabel('temp')
plt.ylabel('Load')
plt.grid()

4. (10 pts.) Fit a multiple regression model by adding to the first model temp, and the temperature squared, the interaction of temperature and hour, and, the interaction of squared temperature and hour. Report the MAPE and r-square.

In [ ]:
df['temp_squared'] = df['temp']**2
df[:5]

In [ ]:
model4 = smf.ols(formula = 'Load ~ year + C(month) + C(day) + C(hour) + C(day_of_week) + \
                                   temp + temp_squared + temp * C(hour) + temp_squared * C(hour)',
                 data = df).fit()

In [ ]:
y4 = df['Load']
X4 = df[['temp','year','month','day','hour','day_of_week','temp_squared']]

In [ ]:
y_pred4 = model4.predict(X4)

In [ ]:
# MAPE
mean_absolute_percentage_error(y4, y_pred4)

In [ ]:
# R-squared
model4.rsquared

5. (10 pts.) Use df['lag24'] = df['Load'].shift(24) to add the Load shifted 24 hours as an additional predictor. Fit the MLR model and report the MAPE and r-square. Remove the NaNs before building the model.

In [ ]:
df5 = df.copy()
df5['lag24'] = df5['Load'].shift(24)
df5[:5]

In [ ]:
df5 = df5.dropna(how = 'any')
df5[:5]

In [ ]:
model5 = smf.ols(formula = 'Load ~ year + C(month) + C(day) + C(hour) + C(day_of_week) + \
                                   temp + temp_squared + temp * C(hour) + temp_squared * C(hour) + \
                                   lag24',
                 data = df5).fit()

In [ ]:
y5 = df5['Load']
X5 = df5[['temp','year','month','day','hour','day_of_week','temp_squared','lag24']]

In [ ]:
y_pred5 = model5.predict(X5)

In [ ]:
# MAPE
mean_absolute_percentage_error(y5, y_pred5)

In [ ]:
# R-squared
model5.rsquared

6. (10 pts.) Use sm.graphics.tsa.plot_pacf(load, lags = 60) to plot partial autocorrelations (adjust the argument lags = 60 as needed). Lags with a large partial autocorrelations (+ or -) should be good predictors. Fit a MLR model with the additional lags found and report the MAPE and r-square.

In [ ]:
pacf = sm.graphics.tsa.plot_pacf(Load, lags = 30)

In [ ]:
# Because most of the forecasted load is bought 14 hours before the start of the “flow date”, 
# lags should be bigger than 14.
# When lag = 24/25/26, the Partial Autocorrelations are large, and those should be good predictors.

In [ ]:
df6 = df.copy()
df6[:5]

In [ ]:
df6['lag24'] = df6['Load'].shift(24)
df6['lag25'] = df6['Load'].shift(25)
df6['lag26'] = df6['Load'].shift(26)
df6['lagaweek'] = df6['Load'].shift(24*7)
df6[:5]

In [ ]:
df6 = df6.dropna(how = 'any')
df6[:5]

In [ ]:
model6 = smf.ols(formula = 'Load ~ year + C(month) + C(day) + C(hour) + C(day_of_week) + \
                                   temp + temp_squared + temp * C(hour) + temp_squared * C(hour) + \
                                   lag24 + lag25 + lag26 + lagaweek',
                 data = df6).fit()

In [ ]:
y6 = df6['Load']
X6 = df6[['temp','year','month','day','hour','day_of_week','temp_squared',
          'lag24','lag25','lag26','lagaweek']]

In [ ]:
y_pred6 = model6.predict(X6)

In [ ]:
# MAPE
mean_absolute_percentage_error(y6, y_pred6)

In [ ]:
# R-squared
model6.rsquared

7. (20 pts.) Split the data into a test set (2019 values) and a train set (all other years). Fit the best model found and report the MAPE and r-square for the test set.

In [ ]:
df7 = df.copy()

In [ ]:
df7['lag24'] = df7['Load'].shift(24)
df7['lag25'] = df7['Load'].shift(25)
df7['lag26'] = df7['Load'].shift(26)
df7['lagaweek'] = df7['Load'].shift(24*7)
df7[:5]

In [ ]:
df7 = df7.dropna(how = 'any')
df7[:5]

In [ ]:
test = df7[df7['year'] == 2019]
train = df7[df7['year'] != 2019]

In [ ]:
model7 = smf.ols(formula = 'Load ~ year + C(month) + C(day) + C(hour) + C(day_of_week) + \
                                   temp + temp_squared + temp * C(hour) + temp_squared * C(hour) + \
                                   lag24 + lag25 + lag26 + lagaweek',
                 data = train).fit()

In [ ]:
y7 = test['Load']
X7 = test[['temp','year','month','day','hour','day_of_week','temp_squared',
          'lag24','lag25','lag26','lagaweek']]

In [ ]:
y_pred7 = model7.predict(X7)

In [ ]:
# MAPE
mean_absolute_percentage_error(y7, y_pred7)

In [ ]:
# R-squared
model7.rsquared

8. (10 pts.) Plot the cumulative load by year and month. This is called a seasonal chart (see below). It is useful to display the seasonality of the data. Adjust the x-axis with your choice.

In [ ]:
df8 = df[df['year'] != 2019]

In [ ]:
df8 = df8.pivot_table(values = 'Load', columns = 'year', index = 'month', aggfunc = sum)
df8[:5]

In [ ]:
df8.plot(figsize = (16,6), grid = True)
plt.ylabel('Monthly Load')
plt.xlim(1,12)
plt.legend()

In [ ]:
# Use the Loop
for i in df8.columns:
    df8.loc[:,i].plot(figsize = (16,6))
plt.grid()
plt.ylabel('Monthly Load')
plt.xlim(1,12)
plt.legend()